#  Modified Triplet Loss : Ungraded Lecture Notebook
In this notebook you'll see how to calculate the full triplet loss, step by step, including the mean negative and the closest negative. You'll also calculate the matrix of similarity scores.

## Background
This is the original triplet loss function:

$\mathcal{L_\mathrm{Original}} = \max{(\mathrm{s}(A,N) -\mathrm{s}(A,P) +\alpha, 0)}$

It can be improved by including the mean negative and the closest negative, to create a new full loss function. The inputs are the Anchor $\mathrm{A}$, Positive $\mathrm{P}$ and Negative $\mathrm{N}$.

$\mathcal{L_\mathrm{1}} = \max{(mean\_neg -\mathrm{s}(A,P)  +\alpha, 0)}$

$\mathcal{L_\mathrm{2}} = \max{(closest\_neg -\mathrm{s}(A,P)  +\alpha, 0)}$

$\mathcal{L_\mathrm{Full}} = \mathcal{L_\mathrm{1}} + \mathcal{L_\mathrm{2}}$

Let me show you what that means exactly, and how to calculate each step.

## Imports

In [2]:
import numpy as np
import tensorflow as tf

2024-01-05 18:48:24.009259: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-05 18:48:24.249592: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-05 18:48:24.249775: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-05 18:48:24.269954: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-05 18:48:24.363922: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-05 18:48:24.367921: I tensorflow/core/platform/cpu_feature_guard.cc:1

## Similarity Scores
The first step is to calculate the matrix of similarity scores using cosine similarity so that you can look up $\mathrm{s}(A,P)$, $\mathrm{s}(A,N)$ as needed for the loss formulas.

### Two Vectors
First I'll show you how to calculate the similarity score, using cosine similarity, for 2 vectors.

$\mathrm{s}(v_1,v_2) = \mathrm{cosine \ similarity}(v_1,v_2) = \frac{v_1 \cdot v_2}{||v_1||~||v_2||}$
* Try changing the values in the second vector to see how it changes the cosine similarity.




In [3]:
def cosine_similarity(l1 : np.array, l2 : np.array):
    numerator = tf.math.reduce_sum(l1 * l2)
    denominator = tf.math.sqrt(tf.math.reduce_sum(l1 * l1) * tf.math.reduce_sum(l2 * l2))
    
    return numerator / denominator

In [4]:
v1 = np.array([1,2,3], dtype=float)
v2 = np.array([1,2,3.5], dtype=float)

print(f"cosine_similarity: {cosine_similarity(v1, v2)}")

cosine_similarity: 0.9974086507360697


### Two Batches of Vectors
Now i'll show you how to calculate the similarity scores, using cosine similarity, for 2 batches of vectors. These are rows of individual vectors, just like in the example above, but stacked vertically into a matrix. They would look like the image below for a batch size (row count) of 4 and embedding size (column count) of 5.

The data is setup so that $v_{1\_1}$ and $v_{2\_1}$ represent duplicate inputs, but they are not duplicates with any other rows in the batch. This means $v_{1\_1}$ and $v_{2\_1}$ (green and green) have more similar vectors than say $v_{1\_1}$ and $v_{2\_2}$ (green and magenta).

I'll show you two different methods for calculating the matrix of similarities from 2 batches of vectors.

<img src = 'images/v1v2_stacked.png' width="width" height="height" style="height:250px;"/>

Setup vector batches

In [5]:
v1_1 = np.array([1.0, 2.0, 3.0])
v1_2 = np.array([9.0, 8.0, 7.0])
v1_3 = np.array([-1.0, -4.0, -2.0])
v1_4 = np.array([1.0, -7.0, 2.0])
v1 = np.vstack([v1_1, v1_2, v1_3, v1_4])
v1

array([[ 1.,  2.,  3.],
       [ 9.,  8.,  7.],
       [-1., -4., -2.],
       [ 1., -7.,  2.]])

In [6]:
v2_1 = v1_1 + np.random.normal(0, 2, 3)
v2_2 = v1_2 + np.random.normal(0, 2, 3)
v2_3 = v1_3 + np.random.normal(0, 2, 3)
v2_4 = v1_4 + np.random.normal(0, 2, 3)

v2 = np.vstack([v2_1, v2_2, v2_3, v2_4])
v2

array([[-0.73588987, -3.16543062,  5.42822526],
       [ 6.99100145, 12.01127535,  8.11570464],
       [-2.01294988, -3.50241275, -3.47638823],
       [ 1.0696933 , -9.91094933, -1.27033772]])

### Similarity Score Calculations

In [7]:
def similarity_scores_1(m1 : np.array, m2 : np.array):
    assert m1.shape == m2.shape
    sim = np.zeros([len(m1), len(m1)])
    
    for row in range(m1.shape[0]):
        for col in range(m1.shape[1]):
            sim[row][col] = cosine_similarity(m2[row], m1[col]).numpy()
    
    return sim

In [8]:
def similarity_scores_2(m1, m2):
    assert m1.shape == m2.shape
    
    def norm(x):
        return tf.math.l2_normalize(x, axis=1)

    return tf.linalg.matmul(norm(m2), norm(m1), transpose_b=True)
    

In [9]:
v1_test = np.ones(shape=(3,3))
v2_test = np.ones(shape=(3,3))
v1_test, v2_test

(array([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]),
 array([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]))

In [10]:
sim_scores_1 = similarity_scores_1(v1_test, v2_test)
sim_scores_2 = similarity_scores_2(v1_test, v2_test)

sim_scores_1, sim_scores_2

(array([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]),
 <tf.Tensor: shape=(3, 3), dtype=float64, numpy=
 array([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])>)

In [11]:

print(f"Both score are correct: {np.allclose(sim_scores_1, sim_scores_2)}")

Both score are correct: True


In [12]:
v1_test[0][1]=2

sim_scores_1 = similarity_scores_1(v1_test, v2_test)
sim_scores_2 = similarity_scores_2(v1_test, v2_test)
sim_scores_1, sim_scores_2

(array([[0.94280904, 1.        , 1.        ],
        [0.94280904, 1.        , 1.        ],
        [0.94280904, 1.        , 1.        ]]),
 <tf.Tensor: shape=(3, 3), dtype=float64, numpy=
 array([[0.94280904, 1.        , 1.        ],
        [0.94280904, 1.        , 1.        ],
        [0.94280904, 1.        , 1.        ]])>)

In [13]:
print(f"Both score are correct: {np.allclose(sim_scores_1, sim_scores_2)}")

Both score are correct: True


In [14]:
sim_scores_1 = similarity_scores_1(v1, v2)
sim_scores_2 = similarity_scores_2(v1, v2)
sim_scores_1, sim_scores_2

(array([[ 0.38939646,  0.06866857,  0.08764871,  0.        ],
        [ 0.91934696,  0.96278932, -0.96632698,  0.        ],
        [-0.9752062 , -0.9493276 ,  0.94072395,  0.        ],
        [-0.60007947, -0.56121865,  0.89281208,  0.        ]]),
 <tf.Tensor: shape=(4, 4), dtype=float64, numpy=
 array([[ 0.38939646,  0.06866857,  0.08764871,  0.69428908],
        [ 0.91934696,  0.96278932, -0.96632698, -0.51457936],
        [-0.9752062 , -0.9493276 ,  0.94072395,  0.39707742],
        [-0.60007947, -0.56121865,  0.89281208,  0.91956166]])>)

In [15]:
print(f"Both score are correct: {np.allclose(sim_scores_1, sim_scores_2)}")

Both score are correct: False


## Hard Negative Mining

I'll now show you how to calculate the mean negative $mean\_neg$ and the closest negative $close\_neg$ used in calculating $\mathcal{L_\mathrm{1}}$ and $\mathcal{L_\mathrm{2}}$.


$\mathcal{L_\mathrm{1}} = \max{(mean\_neg -\mathrm{s}(A,P)  +\alpha, 0)}$

$\mathcal{L_\mathrm{2}} = \max{(closest\_neg -\mathrm{s}(A,P)  +\alpha, 0)}$

You'll do this using the matrix of similarity scores you already know how to make, like the example below for a batch size of 4. The diagonal of the matrix contains all the $\mathrm{s}(A,P)$ values, similarities from duplicate question pairs (aka Positives). This is an important attribute for the calculations to follow.

<img src = 'images/ss_matrix.png' width="width" height="height" style="height:250px;"/>


### Mean Negative
$mean\_neg$ is the average of the off diagonals, the $\mathrm{s}(A,N)$ values, for each row.

### Closest Negative
$closest\_neg$ is the largest off diagonal value, $\mathrm{s}(A,N)$, that is smaller than the diagonal $\mathrm{s}(A,P)$ for each row.
* Try using a different matrix of similarity scores. 

In [16]:
sim_hardcoded = np.array(
    [
        [0.9, -0.8, 0.3, -0.5],
        [-0.4, 0.5, 0.1, -0.1],
        [0.3, 0.1, -0.4, -0.8],
        [-0.5, -0.2, -0.7, 0.5],
    ]
)
sim_hardcoded

array([[ 0.9, -0.8,  0.3, -0.5],
       [-0.4,  0.5,  0.1, -0.1],
       [ 0.3,  0.1, -0.4, -0.8],
       [-0.5, -0.2, -0.7,  0.5]])

In [17]:
sim = sim_hardcoded
b = sim.shape
b

(4, 4)

In [19]:
sim_diag = tf.linalg.diag(sim)
sim_diag

<tf.Tensor: shape=(4, 4, 4), dtype=float64, numpy=
array([[[ 0.9,  0. ,  0. ,  0. ],
        [ 0. , -0.8,  0. ,  0. ],
        [ 0. ,  0. ,  0.3,  0. ],
        [ 0. ,  0. ,  0. , -0.5]],

       [[-0.4,  0. ,  0. ,  0. ],
        [ 0. ,  0.5,  0. ,  0. ],
        [ 0. ,  0. ,  0.1,  0. ],
        [ 0. ,  0. ,  0. , -0.1]],

       [[ 0.3,  0. ,  0. ,  0. ],
        [ 0. ,  0.1,  0. ,  0. ],
        [ 0. ,  0. , -0.4,  0. ],
        [ 0. ,  0. ,  0. , -0.8]],

       [[-0.5,  0. ,  0. ,  0. ],
        [ 0. , -0.2,  0. ,  0. ],
        [ 0. ,  0. , -0.7,  0. ],
        [ 0. ,  0. ,  0. ,  0.5]]])>

In [20]:
sim_diag_part = tf.linalg.diag_part(sim)
sim_diag_part

<tf.Tensor: shape=(4,), dtype=float64, numpy=array([ 0.9,  0.5, -0.4,  0.5])>

In [20]:
def get_mean_negative(m : np.array):
    b = m.shape[0]
    sim_ap = tf.linalg.diag_part(m)
    sim_an = m - tf.linalg.diag(sim_ap)
    return tf.reduce_sum(sim_an, axis=1) / (b - 1)

In [21]:
mean_negative = get_mean_negative(sim)
mean_negative

<tf.Tensor: shape=(4, 4, 4), dtype=float64, numpy=
array([[[ 0.9       , -1.06666667,  0.4       , -0.66666667],
        [-0.53333333,  0.93333333,  0.13333333, -0.13333333],
        [ 0.4       ,  0.13333333, -0.63333333, -1.06666667],
        [-0.66666667, -0.26666667, -0.93333333,  0.83333333]],

       [[ 1.33333333, -1.06666667,  0.4       , -0.66666667],
        [-0.53333333,  0.5       ,  0.13333333, -0.13333333],
        [ 0.4       ,  0.13333333, -0.56666667, -1.06666667],
        [-0.66666667, -0.26666667, -0.93333333,  0.7       ]],

       [[ 1.1       , -1.06666667,  0.4       , -0.66666667],
        [-0.53333333,  0.63333333,  0.13333333, -0.13333333],
        [ 0.4       ,  0.13333333, -0.4       , -1.06666667],
        [-0.66666667, -0.26666667, -0.93333333,  0.93333333]],

       [[ 1.36666667, -1.06666667,  0.4       , -0.66666667],
        [-0.53333333,  0.73333333,  0.13333333, -0.13333333],
        [ 0.4       ,  0.13333333, -0.3       , -1.06666667],
        [-0.6

In [22]:
b = sim.shape[0]
np.identity(b)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [25]:
def get_closest_negative(m : np.array):
    b = m.shape[0]
    sim_ap = tf.linalg.diag_part(m)
    sim_an = m - tf.linalg.diag(sim_ap)
    mask_1 = (tf.eye(b) == 1)
    mask_2 = sim_an > tf.expand_dims(sim_ap, 1)
    mask = tf.cast(mask_1 | mask_2, tf.float64)
    sim_an_masked = sim_an - 2.0 * mask
    return np.max(sim_an_masked, axis=1, keepdims=True)
    

In [26]:
closest_negative = get_closest_negative(sim)
closest_negative

array([[[[-1.1, -0.8, -1.7, -0.5],
         [-0.4, -0.7, -1.9, -0.1],
         [-1.7, -1.9, -2.4, -0.8],
         [-0.5, -0.2, -0.7, -1. ]]],


       [[[-0.7, -0.8, -1.7, -0.5],
         [-0.4, -1.5, -1.9, -0.1],
         [-1.7, -1.9, -2.4, -0.8],
         [-0.5, -0.2, -0.7, -1.4]]],


       [[[-1.1, -0.8, -1.7, -0.5],
         [-0.4, -1.5, -1.9, -0.1],
         [-1.7, -1.9, -2. , -0.8],
         [-0.5, -0.2, -0.7, -0.7]]],


       [[[-0.6, -0.8, -1.7, -0.5],
         [-0.4, -1.3, -1.9, -0.1],
         [-1.7, -1.9, -1.7, -0.8],
         [-0.5, -0.2, -0.7, -1.5]]]])

## The Loss Functions

The last step is to calculate the loss functions.

$\mathcal{L_\mathrm{1}} = \max{(mean\_neg -\mathrm{s}(A,P)  +\alpha, 0)}$

$\mathcal{L_\mathrm{2}} = \max{(closest\_neg -\mathrm{s}(A,P)  +\alpha, 0)}$

$\mathcal{L_\mathrm{Full}} = \mathcal{L_\mathrm{1}} + \mathcal{L_\mathrm{2}}$

In [ ]:
def siamese_loss_function(m : np.array, alpha = 0.25):
    b = m.shape[0]
    mean_neg = get_mean_negative(m)
    closest_neg = get_closest_negative(m)
    sim_ap = np.diag(m)
    
    l1 = tf.maximum(mean_neg - sim_ap.reshape(b, 1) + alpha, 0)
    l2 = tf.maximum(closest_neg - sim_ap.reshape(b, 1) + alpha, 0)
    return l1 + l2

In [ ]:
loss = siamese_loss_function(sim)
loss

array([[0.        ],
       [0.        ],
       [0.51666667],
       [0.        ]])